In [ ]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [1]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

In [3]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/Fine/
  inflating: data/Fine/1715432224001.jpg  
  inflating: data/Fine/1715432224011.jpg  
  inflating: data/Fine/1715432224021.jpg  
  inflating: data/Fine/1715432224031.jpg  
  inflating: data/Fine/1715432224040.jpg  
  inflating: data/Fine/1715432224058.jpg  
  inflating: data/Fine/1715432224074.jpg  
  inflating: data/Fine/1715432224105.jpg  
  inflating: data/Fine/1715432224123.jpg  
  inflating: data/Fine/1715432224136.jpg  
  inflating: data/Fine/fine.69e8dc05-0faf-11ef-8d1d-8c8caaf81959.jpg  
  inflating: data/Fine/fine.6b1cf559-0faf-11ef-b974-8c8caaf81959.jpg  
  inflating: data/Fine/fine.6c503e3d-0faf-11ef-bddf-8c8caaf81959.jpg  
  inflating: data/Fine/fine.6d847b12-0faf-11ef-987f-8c8caaf81959.jpg  
  inflating: data/Fine/fine.6eb85122-0faf-11ef-83b2-8c8caaf81959.jpg  
  inflating: data/Fine/fine.6feb7593-0faf-11ef-b9a3-8c8caaf81959.jpg  
  inflating: data/Fine/fine.712073ea-0faf-11ef-94b4-8c8caaf81959.jpg  
  inflating

In [4]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load dataset
data = datasets.ImageFolder('data/', transform=transform)
print(len(data))

# Split dataset into train and validation
train_size = int(0.8 * len(data))
val_size = len(data) - train_size
train_data, val_data = torch.utils.data.random_split(data, [train_size, val_size])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=4)

184


In [7]:
# Load pre-trained model
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# Replace the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, num_epochs, val_loss/len(val_loader), 100 * correct / total))


Epoch [1/30], Loss: 1.5272, Accuracy: 32.43%
Epoch [2/30], Loss: 2.0571, Accuracy: 27.03%
Epoch [3/30], Loss: 1.0457, Accuracy: 48.65%
Epoch [4/30], Loss: 0.9145, Accuracy: 64.86%
Epoch [5/30], Loss: 0.8120, Accuracy: 67.57%
Epoch [6/30], Loss: 0.7185, Accuracy: 75.68%
Epoch [7/30], Loss: 0.6945, Accuracy: 75.68%
Epoch [8/30], Loss: 0.5794, Accuracy: 81.08%
Epoch [9/30], Loss: 0.5855, Accuracy: 78.38%
Epoch [10/30], Loss: 0.7744, Accuracy: 70.27%
Epoch [11/30], Loss: 0.5917, Accuracy: 72.97%
Epoch [12/30], Loss: 0.6190, Accuracy: 75.68%
Epoch [13/30], Loss: 0.4898, Accuracy: 86.49%
Epoch [14/30], Loss: 0.5610, Accuracy: 75.68%
Epoch [15/30], Loss: 0.4729, Accuracy: 78.38%
Epoch [16/30], Loss: 0.4205, Accuracy: 89.19%
Epoch [17/30], Loss: 0.3602, Accuracy: 86.49%
Epoch [18/30], Loss: 0.4531, Accuracy: 81.08%
Epoch [19/30], Loss: 0.3307, Accuracy: 91.89%
Epoch [20/30], Loss: 0.3644, Accuracy: 86.49%
Epoch [21/30], Loss: 0.3617, Accuracy: 83.78%
Epoch [22/30], Loss: 0.5333, Accuracy: 75.6

In [10]:
torch.save(model.state_dict(), 'model_params.pth')